In [ ]:
using JSON
using LinearAlgebra
using SparseArrays
using StatsBase
using Distributions

# Some conceptual background

Covariance estimation is challenging, and the difficulty increases with the dimension of the data.

However, consider this unusual situation:

* We assume the precision matrix (inverse covariance matrix) \\( \Omega \\) is a linear combination of sparse symmetric positive definite *basis matrices* \\(\Omega_1, \ldots, \Omega_k\\).
* We assume the sparse precision matrices \\(\Omega_1, \ldots, \Omega_K\\) have little off-diagonal "overlap" with each other. I.e., there exist relatively few indices \\( (i,j) \\) such that multiple basis matrices have nonzero entries at \\( (i,j) \\).

In this special case, covariance estimation reduces to finding \\( \alpha_1, \ldots, \alpha_K \\), the coefficients of the linear combination \\( \Omega = \sum_k \alpha_k \Omega_k \\). 

Furthermore: under these circumstances, increasing the dimension of the data actually *improves* our ability to estimate covariance. Each new dimension yields more information about the relative contributions of the different basis matrices.

### Load some data

In [ ]:
pwys = JSON.parsefile("../analyses/temp/pathways/TCGA.json");
data = JSON.parsefile("../analyses/temp/experimental_eval/data/TCGA.json");
pattern = JSON.parsefile("../analyses/temp/experimental_eval/observations/pwys=TCGA__test=0.1__rep=0.json");

### Convert data to matrices: observed and hidden

In [ ]:
data_matrix = transpose(hcat(data["data"]...))
obs_idx = pattern["measured_train"]
observed_data = data_matrix[:, obs_idx]
obs_idx_set = Set(obs_idx)

hidden_idx = [idx for idx=1:size(data_matrix,2) if !( idx in obs_idx_set )]
hidden_data = 0.01 * randn(size(data_matrix,1), size(data_matrix,2) - size(obs_idx,1));

### Convert the pathways to matrices

## (Approximate) Method of Moments

If we try a Maximum Likelihood approach, we end up with this mystifying first-order-sufficient condition:

\\( \sum_{m,n} \left[ (\sum_k \alpha_k \Omega_k )^{-1} \right]_{m,n} \cdot \left[ \Omega_i \right]_{m,n} = x^T \Omega_i x ~ ~ \forall i \in [K] \\)

This is satisfied if 

\\( (\sum_k \alpha_k \Omega_k )^{-1} = x x^T\\).

However, it isn't straightforward to find the \\(\alpha_k \\) satisfying that expression.
Solving it via gradient descent would be prohibitively expensive -- computing the gradient would entail inverting a large matrix at every step.

Alternatively we can convert this to a **Method of Moments** estimator by seeking \\( \alpha_1, \ldots, \alpha_K \\) satisfying 

\\( \sum_k \alpha_k \Omega_k  = \left[\lambda x x^T + (1 - \lambda) \sigma^2 I \right]^{-1} \\)

where \\( \lambda \in [0,1) \\). That is, we make the empirical covariance invertible and set its inverse to a linear combination of the basis matrices.

We can write the inverse on the RHS in closed form:

\\(\frac{1}{\sigma^2 (1 - \lambda)} \left[ I  - \frac{\lambda}{\lambda x^\top x + (1 - \lambda) \sigma^2} x x^\top \right]\\) 


The equation is unlikely to have an exact solution.
However, we can solve it approximately in a least-squares sense.

## Let's simulate some data and see how this goes

In [ ]:
data_dim = 8000
n_pwys = 100
n_patients = 50
pwy_density = 0.002;

In [ ]:
# Function for building a sparse, signed, undirected network (edge list)
function build_sparse_network(data_dim, density)
    n_edges = Int(round(density*data_dim*(data_dim - 1)*0.5))
    edges = Set{Pair{Int64,Int64}}()
    result = zeros(Int64, n_edges, 3)
    row = 1
    while row <= n_edges
        pair = collect(samplepair(data_dim))
        sort!(pair)
        pair_pair = (pair[1] => pair[2])
        if !(pair_pair in edges)
            push!(edges, pair[1] => pair[2])
            sgn = rand([-1,1])
            result[row,:] .= [pair[1], pair[2], sgn]
            row = row + 1
        end
    end
    
    return result
end

In [ ]:
# Build a list ("basis") of sparse, undirected networks
networks = [build_sparse_network(data_dim, pwy_density) for i=1:n_pwys]

In [ ]:
# Generate some "pathway activations"
activations = rand(Distributions.Exponential(1.0), n_pwys)

In [ ]:
# Generate a big fat precision matrix
off_diag_multiplier = 1.0
diag_multiplier = 200.0

function build_precision_matrix(networks, activations, data_dim)
    
    combined_edges = Set{Pair{Int64,Int64}}()
    for net in networks
        for i=1:size(net,1)
            push!(combined_edges, net[i,1] => net[i,2])
        end
    end
    encoder = collect(combined_edges)
    decoder = Dict([edge => i for (i, edge) in enumerate(encoder)])
    
    result_I = [edge.first for edge in encoder]
    result_J = [edge.second for edge in encoder]
    result_V = zeros(length(encoder))
    
    for (i, net) in enumerate(networks)
        for j=1:size(net,1)
            edge = (net[j,1] => net[j,2])
            result_idx = decoder[edge]
            result_V[result_idx] += net[j,3]*activations[i]*off_diag_multiplier
        end
    end
    
    result = Symmetric(sparse(result_I, result_J, result_V, data_dim, data_dim) + sparse(I,data_dim,data_dim)*diag_multiplier)
end


In [ ]:
prec = build_precision_matrix(networks, activations, data_dim)
# prec = prec + 200.0*sparse(I,dim,dim)

In [ ]:
# Sample some data from a MVN, parameterized by the big fat precision matrix
F = cholesky(prec)
z = randn(data_dim, n_patients)

X = F.UP\z

### Build the matrix of precision matrix entries

In [ ]:
function build_regression_features(networks)
    result = Dict{Pair{Int64,Int64},Vector{Float64}}()
    for (i,net) in enumerate(networks)
        for j=1:size(net,1)
            if net[j,1] == net[j,2]
                continue
            end
            edgepair = net[j,1]=>net[j,2]
            if !(edgepair in keys(result))
                result[edgepair] = zeros(size(networks,1))
            end
            result[edgepair][i] = net[j,3]
        end
    end
    
    result_idx = collect(keys(result))
    result_mat = zeros(length(result_idx),size(networks,1))
    for i=1:size(result_mat,1)
        result_mat[i,:] .= result[result_idx[i]]
    end
    return result_idx, result_mat
end

In [ ]:
reg_feat_idx, reg_feat = build_regression_features(networks)

### Compute entries of the empirical precision matrix

In [ ]:
function compute_empirical_prec(i, j, x, lambda, sigma_sq, denom, normsq)
    result =  -lambda * x[i]*x[j] / (lambda*normsq + denom)
    if i == j
        result += 1.0
    end
    return result/denom
end

function build_regression_target(reg_feat_idx, X, lambda, sigma_sq)
    result = zeros(size(reg_feat_idx,1), size(X,2))
    denom = (1.0 - lambda) * sigma_sq
    for j=1:size(X,2)
        normsq = dot(X[:,j],X[:,j])
        big_mult = -lambda /(lambda*normsq + denom)
        for (i,p) in enumerate(reg_feat_idx)
            result[i,j] = big_mult * X[p.first,j] * X[p.second,j]
        end
    end
    return result
end

### Build the vector of precision matrix entries

In [ ]:
lambda = 0.5
sigma_sq = 1.0
B = build_regression_target(reg_feat_idx, X, lambda, sigma_sq)

In [ ]:
Y = transpose(reg_feat) * B
regularized = transpose(reg_feat)*reg_feat + 100000.0*I

In [ ]:
pred = reg_feat \ B

In [ ]:
regularized_pred = regularized \ Y

In [ ]:
scores = [corspearman(pred[:,i], activations) for i=1:n_patients]

In [ ]:
regularized_scores = [corspearman(regularized_pred[:,i], activations) for i=1:n_patients]

In [ ]:
mean(regularized_scores)

In [ ]:
mean(scores)